In [15]:
import requests
from pymongo import MongoClient

In [16]:
# Client connects to "localhost" by default
client = MongoClient("mongodb://localhost:27017/")
# Create local "nobel" database on the fly
db = client["nobel"]

In [17]:
for collection_name in ["prizes"]:
    # collect the data from the API
    response = requests.get("http://api.nobelprize.org/v1/{}.json".format(collection_name[:-1]))
    # convert the data to json
    documents = response.json()[collection_name]
    # Create collections on the fly
    db[collection_name].insert_many(documents)

In [ ]:
prizes_collection = db["prizes"]

In [ ]:
# Count documents in a collection
n_prizes = db.prizes.count_documents(filter)
n_laureates = db.laureates.count_documents(filter)

In [ ]:
# Find one document to inspect
doc = db.prizes.find_one()

In [ ]:
# Count documents by providing afilter document to match
filter_doc = {
    'born': '1845-03-27',
    'diedCountry': 'Germany',
    'gender': 'male',
    'surname': 'Röntgen'
}
db.laureates.count_documents(filter_doc)

In [ ]:
## Simple filters
db.laureates.count_documents({'gender': 'female'})

In [ ]:
db.laureates.count_documents({'diedCountry': 'France'})

In [ ]:
db.laureates.count_documents({'bornCity': 'Warsaw'})

In [ ]:
filter_doc = {'gender': 'female',
'diedCountry': 'France',
'bornCity': 'Warsaw'}
db.laureates.count_documents(filter_doc)
## db.laureates.find_one(filter_doc)

### Query operators

In [ ]:
# Value in a range $in: <list>
db.laureates.count_documents({
    'diedCountry': {
    '$in': ['France', 'USA']
    }
})

In [ ]:
# Not equal $ne : <value>
db.laureates.count_documents({
    'diedCountry': {
    '$ne': 'France'
    }
})

## $ne return null values as well!

In [ ]:
# Comparison: > : $gt , ≥ : $gte | < : $lt , ≤ : $lte
db.laureates.count_documents({
    'diedCountry': {
    '$gt': 'Belgium',
    '$lte': 'USA'
    }
})

In [ ]:
db.laureates.find_one({
"firstname": "Walter",
"surname": "Kohn"})

In [ ]:
a = db.laureates.count_documents({
"prizes.affiliations.name": (
"University of California")})
b = db.laureates.count_documents({
"prizes.affiliations.city": (
"Berkeley, CA")})
print(f'Number of prizes in University of California is {a}. Number of prizes from Berkeley is {b}.')

In [ ]:
# $exists is to determine if a value exists in that field
db.laureates.count_documents({"bornCountry": {"$exists": False}})

In [ ]:
# at least 2 or more prizes won
db.laureates.count_documents({"prizes.1": {"$exists": True}})

In [ ]:
# same as unique in python
db.laureates.distinct("gender")

In [ ]:
db.laureates.distinct("prizes.category")

In [ ]:
# getting unique prize category with prize won with 4 people
db.laureates.distinct("prizes.category", {"prizes.share": '4'})

In [ ]:
# Prize categories with multi-winners
db.laureates.count_documents({"prizes.1": {"$exists": True}})

#lisitng that categories
db.laureates.distinct("prizes.category", {"prizes.1": {"$exists": True}})

In [ ]:
# $elemMatch
db.laureates.count_documents({"prizes": {"$elemMatch":{"category": "physics", "share": "1"}}})

In [ ]:
# Finding a substring with $regex
db.laureates.distinct("bornCountry",
{"bornCountry": {"$regex": "Poland"}})

# Beginning and ending (and escaping)
from bson.regex import Regex
db.laureates.distinct("bornCountry",{"bornCountry": Regex("^Poland")})

db.laureates.distinct(
"bornCountry",{"bornCountry": Regex("^Poland \(now")})

db.laureates.distinct(
"bornCountry",{"bornCountry": Regex("now Poland\)$")})